In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
import json

# Use a service account
cred = credentials.Certificate('crp-backend-server-b67475fc5e44.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

In [15]:
user_ref = db.collection(u'User')

user_df = pd.read_csv('user_db.csv')

for index, row in user_df.iterrows():
    doc_ref = user_ref.document(row['Registration Number'])
    
    regno = row['Registration Number']
    uname = row['Username']
    passw = row['Password']
    email = row['Email']
    phnno = row['Phone Number']
    usert = row['User Type']
    
    doc_ref.set({
        u'registrationNumber': regno,
        u'name': uname,
        u'password': passw,
        u'email': email,
        u'phoneNumber': phnno,
        u'userType': usert
    })

In [8]:
course_ref = db.collection(u'Course')

with open('course_db.json', 'r') as course_db:
    course_data = json.loads(course_db.read())
    
    for code, info in course_data.items():
        doc_ref = course_ref.document(code)
        
        doc_ref.set(info)